In [ ]:
import requests
import numpy as np
from itertools import permutations


def get_distance_matrix(localidades, origem=None, clientes=None, tipo="int"):
    """
    Obtém a matriz de distâncias e tempos entre as localidades usando o OSRM.

    Args:
        localidades (list): Lista de coordenadas no formato [longitude, latitude].
        origem (int): Índice da localidade de origem (opcional).
        clientes (list): Lista de índices das localidades de destino (opcional).
        tipo (str): Tipo de conversão dos valores ("int" ou "float").

    Returns:
        duration_matrix (list): Matriz de tempos em segundos.
        distance_matrix (list): Matriz de distâncias em metros.
    """
    # Formata as coordenadas para o formato "longitude,latitude"
    coordinates = ";".join([f"{loc[0]},{loc[1]}" for loc in localidades])

    # Define as origens e destinos
    if origem is not None:
        sources = [origem]  # Apenas a origem especificada
    else:
        sources = list(range(len(localidades)))  # Todas as localidades como origens

    if clientes is not None:
        destinations = clientes  # Apenas os clientes especificados
    else:
        destinations = list(range(len(localidades)))  # Todas as localidades como destinos

    # Configura os parâmetros da requisição
    params = {
        "sources": ";".join(map(str, sources)),  # Índices das origens
        "destinations": ";".join(map(str, destinations)),  # Índices dos destinos
        "annotations": "duration,distance"  # Solicita tempos e distâncias
    }

    # Faz a requisição ao OSRM
    url = "http://router.project-osrm.org/table/v1/driving/"
    response = requests.get(url + coordinates, params=params)

    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        data = response.json()
        duration_matrix = data['durations']  # Matriz de tempos em segundos
        distance_matrix = data['distances']  # Matriz de distâncias em metros

        # Converte os valores para o tipo especificado
        if tipo == "int":
            duration_matrix = [[int(val) for val in row] for row in duration_matrix]
            distance_matrix = [[int(val) for val in row] for row in distance_matrix]
        elif tipo == "float":
            duration_matrix = [[float(val) for val in row] for row in duration_matrix]
            distance_matrix = [[float(val) for val in row] for row in distance_matrix]
        else:
            raise ValueError("Tipo inválido. Use 'int' ou 'float'.")

        return duration_matrix, distance_matrix
    else:
        print("Erro ao obter a matriz de distâncias:", response.status_code)
        return None, None


def solve_tsp_bruteforce(distance_matrix):
    n = len(distance_matrix)
    best_order = None
    best_cost = float("inf")
    
    for perm in permutations(range(1, n)):
        cost = distance_matrix[0, perm[0]] + sum(
            distance_matrix[perm[i], perm[i+1]] for i in range(len(perm) - 1)
        )
        if cost < best_cost:
            best_cost = cost
            best_order = perm
    return [0] + list(best_order)

origem = [(-49.2733,-25.4284)]

clientes = [
    (-49.2597, -25.3930),  # Jardim Botânico
    (-49.2673, -25.4404),  # Ópera de Arame
    (-49.2670, -25.4230),  # Museu Oscar Niemeyer
    (-49.2830, -25.4170),  # Parque Barigui
    (-49.2943, -25.4093),  # Parque Tanguá
    (-49.2647, -25.3902),  # Bosque Alemão
    (-49.2722, -25.4354),  # Mercado Municipal
    (-49.3027, -25.4503),  # Torre Panorâmica
    (-49.3213, -25.4184),  # Bosque do Papa
    (-49.2767, -25.4442)   # Catedral Basílica Menor de Nossa Senhora da Luz
]

localidades = origem + clientes

# Obtemos as matrizes de tempo e distância
duration_matrix, distance_matrix = get_distance_matrix(localidades)

# Resolvemos o problema do TSP
best_route = solve_tsp_bruteforce(np.array(distance_matrix))

# Cálculo do custo total em km e tempo total em minutos
total_distance = 0  # distância total em metros
total_time = 0  # tempo total em segundos

for i in range(len(best_route) - 1):
    total_distance += distance_matrix[best_route[i]][best_route[i + 1]]
    total_time += duration_matrix[best_route[i]][best_route[i + 1]]

# Convertendo para quilômetros e minutos
total_distance_km = total_distance / 1000  # Convertendo metros para quilômetros
total_time_minutes = total_time / 60  # Convertendo segundos para minutos

print(f'Rota Coords: {best_route}\nDistância total: {total_distance_km:.3f} km\nTempo total: {total_time_minutes:.3f} minutos\n')

Rota Coords: [0, 4, 5, 9, 8, 10, 2, 7, 3, 1, 6]
Distância total: 29.455 km
Tempo total: 51.033 minutos



In [19]:
def get_route(coordinates):
    """
    Obtém a rota real entre uma lista de coordenadas usando o OSRM.
    Retorna uma lista de coordenadas (latitude, longitude) que representam a rota.
    """
    url = "http://router.project-osrm.org/route/v1/driving/"
    coordinates_str = ";".join(coordinates)
    params = {
        "overview": "full",  # Retorna a geometria completa da rota
        "geometries": "geojson"  # Formato das coordenadas
    }
    response = requests.get(url + coordinates_str, params=params)
    if response.status_code == 200:
        data = response.json()
        if data['code'] == 'Ok':
            return data['routes'][0]['geometry']['coordinates']  # Lista de coordenadas (longitude, latitude)
    return None

In [20]:
import folium
from folium.plugins import PolyLineTextPath


# Cria um mapa centrado no Centro de Curitiba
m = folium.Map(location=[-25.4284, -49.2733], zoom_start=12)

inv_localidades = [(lat,lon) for lon, lat in localidades]
inv_clientes = [(lat,lon) for lon, lat in clientes]
inv_origem = [(lat,lon) for lon, lat in origem]

# Adiciona a origem ao mapa
for i, (lat, lon) in enumerate(inv_origem):
    folium.Marker(
        [lat, lon],
        tooltip=f'Centro de Curitiba {i+1}',
        icon=folium.Icon(color='green')
    ).add_to(m)

# Adiciona os clientes ao mapa
for i, (lat, lon) in enumerate(inv_clientes):
    folium.Marker(
        [lat, lon],
        tooltip=f'Cliente {i+1}'
    ).add_to(m)

best_route_coords = [localidades[i] for i in best_route]
route_coords = [f"{lon},{lat}" for lon, lat in best_route_coords]

route = get_route(route_coords)
if route:
    # Converte as coordenadas para o formato [latitude, longitude]
    route_lat_lon = [[coord[1], coord[0]] for coord in route]

    # Cria a linha da rota
    linha = folium.PolyLine(
        route_lat_lon,
        color='blue',
        weight=1,
        opacity=1
    ).add_to(m)

    # Adiciona setas ao longo da linha para indicar o sentido
    PolyLineTextPath(
        linha,
        text='►',  # Seta Unicode
        repeat=True,
        offset=3,
        attributes={'fill': 'blue', 'font-size': '12', 'letter-spacing': '20'}
    ).add_to(m)
m.save('rota_otimizada.html')
print("Mapa salvo como 'rota_otimizada.html'")

Mapa salvo como 'rota_otimizada.html'
